# EthicCompanion: Gemma 3n Fine-tuning for Hackathon

Welcome to the **EthicCompanion Gemma 3n Implementation** for the Google Gemma 3n Impact Challenge! 

## 🎯 Project Overview
EthicCompanion leverages **Gemma 3n's multimodal capabilities** to help users navigate information overload with ethical clarity and inner peace.

## 🚀 Gemma 3n Advantages for EthicCompanion
- **Multimodal Input**: Text, image, video, and audio processing
- **Efficient Architecture**: 2B/4B effective parameters with selective activation
- **Ethical Foundation**: Built with responsible AI principles
- **32K Context Window**: Perfect for comprehensive ethical guidance
- **140+ Languages**: Global accessibility for ethical support

## 📊 Models We'll Use
1. **Gemma 3n E2B** (2B effective parameters) - Fast ethical classifications
2. **Gemma 3n E4B** (4B effective parameters) - Complex ethical reasoning
3. **Custom Fine-tuned Models** - Specialized for ethical guidance scenarios

## 🎭 Key Features for Hackathon
- **Crisis Detection**: Identify users needing immediate support
- **Information Overload Management**: Help process overwhelming news
- **Ethical Action Recommendations**: Suggest constructive responses
- **Multimodal Understanding**: Process text, images, and audio content

In [ ]:
# Install Gemma 3n specific requirements for the hackathon
!pip install -q kagglehub
!pip install -q transformers>=4.53.0  # Gemma 3n requires transformers 4.53.0+
!pip install -q timm  # Required for Gemma 3n image processing
!pip install -q torch torchvision torchaudio
!pip install -q accelerate
!pip install -q datasets
!pip install -q peft  # For LoRA fine-tuning
!pip install -q bitsandbytes  # For quantization
!pip install -q google-cloud-aiplatform
!pip install -q google-cloud-storage
!pip install -q pillow requests  # For image processing

print("✅ Gemma 3n environment setup complete!")
print("📦 Key capabilities installed:")
print("- Gemma 3n transformers (4.53.0+)")
print("- Multimodal processing (text, image, audio)")
print("- LoRA fine-tuning capabilities")
print("- Google Cloud integration")

In [ ]:
import os
import json
import kagglehub
import torch
import numpy as np
from datetime import datetime
from PIL import Image
import requests

# Gemma 3n specific imports
from transformers import (
    AutoProcessor, 
    Gemma3nForConditionalGeneration,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    pipeline
)

# Google Cloud imports
from google.cloud import aiplatform
from google.cloud import storage

# LoRA and fine-tuning imports
from peft import LoraConfig, get_peft_model, TaskType

# Data processing
import pandas as pd
from datasets import Dataset

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# Gemma 3n model configurations for hackathon
GEMMA_3N_MODELS = {
    "e2b": {
        "name": "google/gemma-3n-e2b",
        "effective_params": "2B",
        "description": "Efficient model for fast ethical classifications",
        "use_case": "Quick content moderation and ethical scope detection"
    },
    "e4b": {
        "name": "google/gemma-3n-e4b", 
        "effective_params": "4B",
        "description": "Advanced model for complex ethical reasoning",
        "use_case": "Deep ethical analysis and guidance generation"
    }
}

print("🤖 Gemma 3n Models Available:")
for key, model in GEMMA_3N_MODELS.items():
    print(f"  {key.upper()}: {model['name']} ({model['effective_params']} params)")
    print(f"    Use: {model['use_case']}")

print("\n✅ Environment ready for Gemma 3n fine-tuning!")

## 🔧 Configuration and Setup

In [ ]:
# Configuration
CONFIG = {
    "project_id": "your-project-id",  # Replace with your GCP project ID
    "region": "us-central1",
    "model_name": "google/gemma-2b-it",  # Base model to fine-tune
    "dataset_path": "./ethical_guidance_dataset.jsonl",
    "output_dir": "./ethiccompanion-gemma-finetuned",
    "wandb_project": "ethiccompanion-training",
    "max_length": 512,
    "batch_size": 4,
    "learning_rate": 2e-4,
    "num_epochs": 3,
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.1
}

# Initialize Vertex AI
aiplatform.init(project=CONFIG["project_id"], location=CONFIG["region"])

# Initialize wandb for experiment tracking
wandb.init(project=CONFIG["wandb_project"], config=CONFIG)

## 📊 Dataset Preparation

Let's create a high-quality dataset for ethical guidance training.

In [ ]:
# Create ethical guidance training dataset
def create_ethical_dataset():
    """
    Create a structured dataset for ethical guidance training
    """
    
    # Sample training data - in practice, you'd have much more data
    training_conversations = [
        {
            "scenario": "information_overload",
            "user_input": "I feel overwhelmed by constant news about conflicts. How can I manage this?",
            "ethical_response": "I understand feeling overwhelmed by distressing news is very common and shows your empathy. Here are some ethical approaches to manage information overload: 1) Set specific times for news consumption (like 15 minutes morning and evening) to stay informed without constant exposure. 2) Choose 2-3 trusted, factual news sources rather than consuming from many outlets. 3) Practice the 'constructive consumption' approach - for every piece of distressing news, look for one constructive action you can take or positive story. 4) Use mindfulness techniques like deep breathing when you feel overwhelmed. 5) Remember that staying mentally healthy allows you to help others more effectively. Would you like specific mindfulness techniques or suggestions for constructive actions?",
            "reasoning": "Response acknowledges emotions, provides practical steps, emphasizes balance between staying informed and mental health."
        },
        {
            "scenario": "ethical_decision",
            "user_input": "I saw disturbing news and want to help, but I don't know how. What are ethical ways to respond?",
            "ethical_response": "Your desire to help shows compassion and ethical awareness. Here are constructive ways to respond to disturbing news: 1) Research reputable organizations working on the issue and consider donating if you're able. 2) Volunteer your time or skills to local organizations addressing similar issues. 3) Share verified, factual information (not unverified content) to raise awareness responsibly. 4) Engage in respectful dialogue with others to promote understanding. 5) Practice 'informed advocacy' - educate yourself deeply before taking action. 6) Support ethical journalism by subscribing to quality news sources. 7) Take care of your mental health so you can sustain your efforts. Remember, even small ethical actions contribute to positive change.",
            "reasoning": "Provides multiple constructive options, emphasizes verification and sustainability of efforts."
        },
        {
            "scenario": "peace_techniques",
            "user_input": "I need techniques to find inner peace when reading distressing news.",
            "ethical_response": "Finding inner peace while staying engaged with important issues is a valuable skill. Try these evidence-based techniques: 1) **Box Breathing**: Inhale for 4 counts, hold for 4, exhale for 4, hold for 4. Repeat 4-5 times. 2) **5-4-3-2-1 Grounding**: Notice 5 things you see, 4 you can touch, 3 you hear, 2 you smell, 1 you taste. 3) **Compassionate Reframing**: Instead of 'I can't handle this,' try 'This is difficult, and my concern shows I care.' 4) **Constructive Pause**: After reading distressing news, take 5 minutes to think of one positive action before continuing. 5) **Loving-kindness meditation**: Send good wishes to those suffering and to yourself. These techniques help you stay emotionally regulated while maintaining your capacity for empathy and action.",
            "reasoning": "Provides specific, actionable techniques with clear instructions and explains their purpose."
        },
        {
            "scenario": "crisis_detection",
            "user_input": "I can't stop thinking about global problems and it's affecting my sleep.",
            "ethical_response": "I'm concerned about how this is affecting your well-being. When global issues significantly impact sleep and daily functioning, it's important to take this seriously. Here's what I recommend: 1) **Immediate steps**: Establish a 'news curfew' 2 hours before bedtime, practice relaxation techniques, and limit news consumption to specific times. 2) **Professional support**: Consider speaking with a counselor or therapist who can help you process these feelings and develop coping strategies. 3) **Reframe your concern**: Your care about global issues shows empathy, but you can only help effectively if you're mentally healthy. 4) **Channel worry into action**: Choose one specific way to contribute (volunteering, donating, advocacy) rather than feeling helpless. If sleep problems persist or you feel hopeless, please reach out to a mental health professional. Your well-being matters.",
            "reasoning": "Shows serious concern for user's well-being, provides immediate and long-term strategies, emphasizes professional help when needed."
        }
    ]
    
    # Format for training
    formatted_data = []
    for conv in training_conversations:
        # Create instruction-following format
        instruction = f"""You are EthicCompanion, an AI assistant focused on providing ethical guidance for information management and finding inner peace.

User concern: {conv['user_input']}

Provide empathetic, ethical guidance that includes:
1. Acknowledgment of their feelings
2. Practical, actionable steps
3. Mindfulness techniques when appropriate
4. When to seek professional help if needed

Response:"""
        
        formatted_data.append({
            "instruction": instruction,
            "output": conv['ethical_response'],
            "scenario": conv['scenario']
        })
    
    return formatted_data

# Create the dataset
training_data = create_ethical_dataset()

# Save to file
with open(CONFIG["dataset_path"], 'w') as f:
    for item in training_data:
        f.write(json.dumps(item) + '\n')

print(f"✅ Created dataset with {len(training_data)} training examples")
print(f"💾 Saved to: {CONFIG['dataset_path']}")

## 🤖 Model Loading and LoRA Configuration

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"])
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model with 4-bit quantization for efficient training
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name"],
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
)

# Configure LoRA for parameter-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

print(f"✅ Loaded model: {CONFIG['model_name']}")
print(f"🔧 Applied LoRA configuration")
print(f"📊 Trainable parameters: {model.get_nb_trainable_parameters()}")

## 📊 Data Processing and Tokenization

In [ ]:
def tokenize_function(examples):
    """
    Tokenize the training data
    """
    # Combine instruction and output for causal language modeling
    texts = [inst + output for inst, output in zip(examples["instruction"], examples["output"])]
    
    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=CONFIG["max_length"],
        return_tensors="pt"
    )
    
    # For causal LM, labels are the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

# Load and process dataset
dataset = load_dataset("json", data_files=CONFIG["dataset_path"])
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split into train/validation
train_dataset = tokenized_dataset["train"]

print(f"✅ Processed {len(train_dataset)} training examples")
print(f"📏 Max sequence length: {CONFIG['max_length']}")

## 🏋️‍♂️ Training Configuration and Execution

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=CONFIG["output_dir"],
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=4,
    learning_rate=CONFIG["learning_rate"],
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="no",  # No validation set for this simple example
    fp16=True,
    gradient_checkpointing=True,
    remove_unused_columns=False,
    report_to="wandb",
    run_name=f"ethiccompanion-gemma-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

print("🏋️‍♂️ Training configuration ready!")
print(f"📊 Training examples: {len(train_dataset)}")
print(f"⏱️ Estimated training time: {CONFIG['num_epochs']} epochs")

In [ ]:
# Start training
print("🚀 Starting training...")
trainer.train()

# Save the final model
trainer.save_model()
tokenizer.save_pretrained(CONFIG["output_dir"])

print("✅ Training completed!")
print(f"💾 Model saved to: {CONFIG['output_dir']}")

## 🧪 Model Testing and Evaluation

In [ ]:
def test_fine_tuned_model(prompt: str, max_length: int = 200) -> str:
    """
    Test the fine-tuned model with a prompt
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Remove the input prompt from the response
    response = response[len(prompt):].strip()
    
    return response

# Test scenarios
test_prompts = [
    """You are EthicCompanion, an AI assistant focused on providing ethical guidance for information management and finding inner peace.

User concern: I'm feeling anxious about climate change news. How can I stay informed without feeling hopeless?

Provide empathetic, ethical guidance that includes:
1. Acknowledgment of their feelings
2. Practical, actionable steps
3. Mindfulness techniques when appropriate
4. When to seek professional help if needed

Response:""",
    
    """You are EthicCompanion, an AI assistant focused on providing ethical guidance for information management and finding inner peace.

User concern: How do I know if news is reliable? I'm worried about spreading misinformation.

Provide empathetic, ethical guidance that includes:
1. Acknowledgment of their feelings
2. Practical, actionable steps
3. Mindfulness techniques when appropriate
4. When to seek professional help if needed

Response:"""
]

print("🧪 Testing fine-tuned model:")
print("=" * 60)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n**Test {i}:**")
    print(f"**Prompt:** {prompt.split('User concern:')[1].split('Provide empathetic')[0].strip()}")
    
    response = test_fine_tuned_model(prompt)
    print(f"**Response:** {response}")
    print("-" * 40)

## 🚀 Deployment to Vertex AI

In [ ]:
# TODO: Implement deployment to Vertex AI endpoint
# This would involve:
# 1. Converting the model to a format suitable for Vertex AI
# 2. Creating a custom prediction container
# 3. Deploying to a Vertex AI endpoint
# 4. Testing the deployed endpoint

print("🚀 Deployment section - TODO")
print("Next steps:")
print("1. Convert model to Vertex AI compatible format")
print("2. Create custom prediction container")
print("3. Deploy to Vertex AI endpoint")
print("4. Test deployed endpoint")

## 📊 Training Summary and Next Steps

Congratulations! You've successfully fine-tuned a Gemma model for ethical guidance.

### What you accomplished:
- ✅ Created a structured ethical guidance dataset
- ✅ Fine-tuned Gemma using LoRA for parameter efficiency
- ✅ Tracked training with Weights & Biases
- ✅ Tested the fine-tuned model
- ✅ Saved the model for deployment

### Next Steps:
1. **Expand Dataset**: Create a larger, more diverse training dataset
2. **Hyperparameter Tuning**: Experiment with different learning rates, batch sizes
3. **Evaluation Metrics**: Implement comprehensive evaluation metrics
4. **A/B Testing**: Compare with base models in production
5. **Continuous Learning**: Implement feedback loops for model improvement

### Integration with EthicCompanion:
- Update your `llm_service_enhanced.py` to use this fine-tuned model
- Implement model versioning and A/B testing
- Monitor performance in production